# cell

> Class representing a single code cell
- order: 13

In [ ]:
#| default_exp cell
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from nbstata.stata_session import StataSession
from nbstata.magics import StataMagics
from fastcore.basics import patch_to

In [ ]:
#| hide
#| export
magic_handler = StataMagics()

In [ ]:
#| export
class Cell:
    """A class for managing execution of a single code cell"""                  
    def __init__(self, kernel, code_w_magics, silent=False):
        self.noecho = kernel.nbstata_config.noecho
        self.echo = kernel.nbstata_config.echo
        self.quietly = silent
        self.stata_session = kernel.stata_session
        self.code = magic_handler.magic(code_w_magics, kernel, self)
       
    def run(self):
        if not self.code:
            return
        self.stata_session.dispatch_run(self.code, 
            quietly=self.quietly, echo=self.echo, noecho=self.noecho)

Some `Cell` functionality can be tested apart from a kernel:

In [ ]:
from nbstata.config import launch_stata, Config
from fastcore.test import test_eq
from textwrap import dedent
from unittest.mock import Mock

In [ ]:
kernel1 = Mock()
kernel1.nbstata_config = Config()

code_w_magics = '''disp "test output"'''
cell1 = Cell(kernel1, code_w_magics)
cell1.code

'disp "test output"'

In [ ]:
#| eval: false
launch_stata(splash=False)
kernel1.stata_session = StataSession()
cell1a = Cell(kernel1, code_w_magics)
cell1a.run()

test output


In [ ]:
code_w_magics = dedent('''\
    *%%quietly
    disp "test output"
    ''')
cell2 = Cell(kernel1, code_w_magics)
test_eq(cell2.quietly, True)

In [ ]:
#| eval: false
cell2.run()

In [ ]:
kernel1.nbstata_config.env['echo'] = 'True'

code_w_magics = '''disp "test output"'''
cell3 = Cell(kernel1, code_w_magics)
test_eq(cell3.noecho, False)

In [ ]:
#| eval: false
cell3.run()

. disp "test output"
test output


In [ ]:
code_w_magics = dedent('''\
    *%%noecho
    #delimit cr
    disp "test output"
    ''')
cell4 = Cell(kernel1, code_w_magics)
test_eq(cell4.noecho, True)

In [ ]:
#| eval: false
cell4.run()

test output


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()